In [ ]:
import os
import pandas as pd
import json
import numpy as np
import pickle
import ast
from fuzzywuzzy import fuzz,process
import functools
import os
pd.set_option('display.max_columns', None)

In [ ]:
# Ruta de los archivos Yelp
Ruta_archivos_Yelp = '../../Data/yelp/'

## 1 ETL de Archivos Yelp

### 1.1 Archivo checkin.json

In [ ]:
#Funcion para abrir chekin de yelp

def abrir_Archivo_json(archivo):
    merged_data = []  # Lista para almacenar los objetos JSON combinados

    with open(archivo) as file:
        for line in file:
            try:
                obj = json.loads(line)
                merged_data.append(obj)
            except json.JSONDecodeError as e:
                print(f"Error al decodificar JSON en el archivo {archivo}: {str(e)}")

    df = pd.DataFrame(merged_data)  # Crear DataFrame a partir de los objetos JSON
    return df

# Ejemplo de uso
archivo = os.path.join(Ruta_archivos_Yelp, 'checkin.json')
df_checkin_yelp = abrir_Archivo_json(archivo)

In [ ]:
# Revisamos si tiene nulos
df_checkin_yelp.isnull().sum()

In [ ]:
# Revisamos si tiene duplicados
df_checkin_yelp.duplicated().sum()

In [ ]:
# Desanidamos la fecha (en Fecha y hora)
desanidados = []

for _, row in df_checkin_yelp.iterrows():
    business_id = row['business_id']
    dates = row['date'].split(', ')
    for date in dates:
        date, time = date.split(' ')
        desanidados.append([business_id, time, date])

df_desanidado = pd.DataFrame(desanidados, columns=['business_id', 'hour', 'date'])

In [ ]:
# Definimos el campo 'date' como datetime y agregamos los campos year y month
df_desanidado['date'] = pd.to_datetime(df_desanidado['date'])
df_desanidado['year'] = df_desanidado['date'].dt.year
df_desanidado['month'] = df_desanidado['date'].dt.month


In [ ]:
# Preview
df_desanidado.head(2)

In [ ]:
# Exportamos a CSV
#df_desanidado.to_csv('../Salidas/Dataset_Checkin.csv', index=False)

In [ ]:
# Exportamos a PARQUET
#df_desanidado.to_parquet('../../Data/data_procesada/Dataset_Checkin.parquet', index=False)

### 1.2 Archivo user.parquet

In [ ]:
# Leemos el archivo
archivo = os.path.join(Ruta_archivos_Yelp, "user.parquet")
df_User_yelp = pd.read_parquet(archivo)

In [ ]:
# Eliminamos duplicados y reindexamos
df = df_User_yelp.drop_duplicates(subset=["user_id", "name"], keep="first")
df_User_yelp = df_User_yelp.reindex(df.index)
df_User_yelp['id_user'] = df.index

In [ ]:
# Preview
df_User_yelp.head(2)

In [ ]:
# Creamos y exportamos un nuevo dataframe para conservar id_user original con su user_id , el cual será usado para unir tablas
dfusuario = df_User_yelp.loc[:, ["id_user",'user_id']]

In [ ]:
#Renombramos la columna user_id
dfusuario.rename(columns={"user_id": "yelp_id"}, inplace=True)

In [ ]:
# Preview
dfusuario.head(2)

In [ ]:
#Exportamos a CSV
#dfusuario.to_csv("../Salidas/UsuarioYelp.csv", index=False)

In [ ]:
# Eliminamos la columna user_id del dataframe df_user_yelp
df_User_yelp.drop(columns=['user_id'], inplace=True)

In [ ]:
# Aplicamos transformaciones a los tipos de datos en las diferentes variables

# Cambiamos a string la variable name
df_User_yelp['name'] = df['name'].astype(str)

# Cambiamos a tipo datetime la variable yelping_since
df_User_yelp['yelping_since'] = pd.to_datetime(df['yelping_since'])

# Creamos una columna año a partir de elite que devuelva una lista de años separado por comas
df_User_yelp['years'] = df['elite'].str.split(',')

# Creamos un nuevo DataFrame con filas individuales para cada año y usuario
years_df = df_User_yelp[['id_user', 'years']].explode('years')

# Reemplazamos valores vacios por np.nan en years
years_df['years'] = years_df['years'].replace("", np.nan)

# Reemplazamos 20 po 2020 en years
years_df['years'] = years_df['years'].replace("20", "2020")

In [ ]:
# Conventimos el campo years a int convirtiendo valores no válidos a NaN
years_df['years'] = pd.to_numeric(years_df['years'], errors='coerce').astype('Int64')

In [ ]:
years_df.head(2)

In [ ]:
# Exportamos a CSV
#years_df.to_csv("../Salidas/Dataset_User_Elite.csv", index=False)

In [ ]:
# Exportamos a PARQUET
#years_df.to_parquet("../../Data/data_procesada/Dataset_User_Elite.parquet")

In [ ]:
# Eliminamos las columnas elite y years del dataframe original user.parquet
df_User_yelp.drop(columns=['elite'], inplace=True)
df_User_yelp.drop(columns=['years'], inplace=True)

## 1.3 Archivo business.pkl

In [ ]:
# Leemos el archivo
archivo = os.path.join(Ruta_archivos_Yelp, 'business.pkl')
df_business= pd.read_pickle(archivo)

In [ ]:
# Agregamos nuevas columnas al dataframe con valores iniciales en NONE
df_business['NAME']=None
df_business['REVIEW_COUNT']=None
df_business['POSTAL_CODE']=None
df_business['CITY']=None
df_business['STATE']=None
df_business['BUSINESS_ID']=None
df_business['ADDRESS']=None
df_business['LATITUDE']=None
df_business['LONGITUDE']=None
df_business['STARS']=None
df_business['IS_OPEN']=None
df_business['ATTRIBUTES']=None
df_business['CATEGORIES']=None
df_business['HOURS']=None

In [ ]:
# Obtenemos la actualización de la columna "NAME" del DataFrame df_business, donde se fusionan las cadenas de texto presentes en cada valor, eliminando los caracteres no-alfabéticos.
for index,i in enumerate(df_business.name.values):
    arr=[]
    for e in i:
        if isinstance(e,str):
         arr.append(e)
    df_business.loc[index, 'NAME'] = ''.join(arr)

In [ ]:
# Obtenemos la actualización de la columna "CITY" del DataFrame df_business, donde se fusionan las cadenas de texto presentes en cada valor, eliminando los caracteres no-alfabéticos.
for index,i in enumerate(df_business.city.values):
    arr=[]
    for e in i:
        if isinstance(e,str):
         arr.append(e)
    df_business.loc[index, 'CITY'] = ''.join(arr)

In [ ]:
# Obtenemos la actualización de la columna "STATE" del DataFrame df_business, donde se fusionan las cadenas de texto presentes en cada valor, eliminando los caracteres no-alfabéticos.
for index,i in enumerate(df_business.state.values):
    arr=[]
    for e in i:
        if isinstance(e,str):
         arr.append(e)
    df_business.loc[index, 'STATE'] = ''.join(arr)

In [ ]:
# En este script, se actualiza la columna 'REVIEW_COUNT' del DataFrame df_business con el primer número entero encontrado en cada valor, recorriendo los valores de la columna y almacenando los números enteros en una lista antes de asignarlos a la columna.
for index,i in enumerate(df_business.review_count.values):
    arr=[]
    for e in i:
        if isinstance(e,int):
         arr.append(e)
    df_business.loc[index, 'REVIEW_COUNT'] = arr[0]

In [ ]:
# Se fusionan los caracteres de texto presentes en cada valor de la columna 'business_id' del DataFrame df_b, actualizando la columna "BUSINESS_ID" con los valores resultantes
for index,i in enumerate(df_business.business_id.values):
    arr=[]
    for e in i:
        if isinstance(e,str):
         arr.append(e)
    df_business.loc[index, 'BUSINESS_ID'] = ''.join(arr)

In [ ]:
# Se fusionan los caracteres de texto presentes en cada valor de la columna 'address' del DataFrame 'df_business', actualizando la columna 'ADDRESS' con los valores resultantes
for index,i in enumerate(df_business.address.values):
    arr=[]
    for e in i:
        if isinstance(e,str):
         arr.append(e)
    df_business.loc[index, 'ADDRESS'] = ''.join(arr)

In [ ]:
# Se fusionan los caracteres de texto presentes en cada valor de la columna 'postal_code' del DataFrame 'df_business', actualizando la columna 'POSTAL_CODE' con los valores resultantes
for index,i in enumerate(df_business.postal_code.values):
    arr=[]
    for e in i:
        if isinstance(e,str):
         arr.append(e)
    df_business.loc[index, 'POSTAL_CODE'] = ''.join(arr)

In [ ]:
# Se filtran los elementos numéricos mayores que 1 en cada valor de la columna 'latitude' del DataFrame 'df_business'. A continuación, se asigna el primer elemento filtrado a la columna 'LATITUDE' en el DataFrame 'df_business' en la fila correspondiente al índice actual.
for index,i in enumerate(df_business.latitude.values):
    arr=[]
    for e in i:
       if e>1:
         arr.append(e)
    df_business.loc[index, 'LATITUDE'] = arr[0]

In [ ]:
# Se filtran los elementos numéricos menores que -1 en cada valor de la columna 'longitude' del DataFrame 'df_business'. Luego, se asigna el primer elemento filtrado a la columna 'LONGITUDE' en el DataFrame 'df_business' en la fila correspondiente al índice actual.
for index,i in enumerate(df_business.longitude.values):
    arr=[]
    for e in i:
        if e<-1:
            arr.append(e)
    df_business.loc[index, 'LONGITUDE'] = arr[0]

In [ ]:
# Se filtran los elementos numéricos mayores que 0.1 en cada valor de la columna 'stars' del DataFrame 'df_business'. Después, se asigna el primer elemento filtrado a la columna 'STARS' en el DataFrame 'df_business' en la fila correspondiente al índice actual.
for index,i in enumerate(df_business.stars.values):
    arr=[]
    for e in i:
       if e>0.1:
         arr.append(e)
    df_business.loc[index, 'STARS'] = arr[0]

In [ ]:
# Se filtran los elementos numéricos mayores o iguales a 0 en cada valor de la columna 'is_open' del DataFrame 'df_business'. A continuación, se asigna el primer elemento filtrado a la columna 'IS_OPEN' en el DataFrame 'df_business' en la fila correspondiente al índice actual
for index,i in enumerate(df_business.is_open.values):
    arr=[]
    for e in i:
       if e >=0:
         arr.append(e)
    df_business.loc[index, 'IS_OPEN'] = arr[0]

In [ ]:
# Este código busca y guarda el primer diccionario encontrado en la columna 'attributes' del dataframe 'df_business' en la columna 'ATTRIBUTES'.
# No se puede evitar los warnings porque por alguna extraña razón df_business.loc[index,'ATTRIBUTES']=arr[0] no funciona
for index,i in enumerate(df_business.attributes.values):
    arr=[]
    for e in i:
        if isinstance(e,dict):
            arr.append(e)
    if len(arr)>0:
        df_business['ATTRIBUTES'][index] = arr[0]

In [ ]:
# Este código filtra los elementos de tipo cadena en la columna 'categories' del dataframe 'df_business'. Luego, fusiona todas las cadenas filtradas en una sola cadena y la asigna a la columna 'CATEGORIES' en el dataframe 'df_business'.
for index,i in enumerate(df_business.categories.values):
    arr=[]
    for e in i:
        if isinstance(e,str):
         arr.append(e)
    df_business.loc[index, 'CATEGORIES'] = ''.join(arr)

In [ ]:
# En este código, se recorren los valores de la columna 'hours' en el dataframe 'df_business' y se filtran los elementos que son diccionarios. Luego, se asigna el primer diccionario encontrado a la columna 'HOURS' en el dataframe 'df_business' en la fila correspondiente. En resumen, el código extrae y guarda el primer diccionario encontrado en la columna 'hours' del dataframe 'df_business' en la columna 'HOURS'.
# No se puede evitar los warnings porque por alguna extraña razón df_business.loc[index,'HOURS']=arr[0] no funciona
for index,i in enumerate(df_business.hours.values):
    arr=[]
    for e in i:
        if isinstance(e,dict):
            arr.append(e)
    if len(arr)>0:     
        df_business['HOURS'][index] = arr[0]

In [ ]:
# Filtramos el dataframe solo a las columnas nuevas generadas con los bucles, con los nombres iniciales
df_business=df_business[['BUSINESS_ID','NAME','REVIEW_COUNT','CITY','STATE','ADDRESS','POSTAL_CODE','LATITUDE','LONGITUDE','STARS','IS_OPEN','ATTRIBUTES','CATEGORIES','HOURS']]

In [ ]:
df_business.rename(columns = {'BUSINESS_ID':'business_id',
                              'NAME':'name',
                              'ADDRESS':'address',
                              'CITY':'city',
                              'STATE':'state',
                              'POSTAL_CODE':'postal_code',
                              'LATITUDE':'latitude',
                              'LONGITUDE':'longitude',
                              'STARS':'stars',
                              'REVIEW_COUNT':'review_count',
                              'IS_OPEN':'is_open',
                              'ATTRIBUTES':'attributes',
                              'CATEGORIES':'categories',
                              'HOURS':'hours'}, inplace = True)
df_business = df_business[['business_id', 'name', 'address', 'city', 'state', 'postal_code',
       'latitude', 'longitude', 'stars', 'review_count', 'is_open',
       'attributes', 'categories', 'hours']]

In [ ]:
df_business2 = df_business.copy()
df_business2.head(2)

In [ ]:
# Cargamos un nuevo CSV con las ciudades de Estados Unidos
# Fuente: https://simplemaps.com/data/us-cities
city = pd.read_csv('../../Data/data_extra/uscities.csv')
city.head(2)

In [ ]:
#ciudades unicas de estados unidos
ciudades_estados_unidos = city['city'].unique()

In [ ]:
# la primera función busca la mejor coincidencia de una ciudad en una lista, mientras que la segunda función calcula un umbral de similitud basado en la longitud de una ciudad. Ambas funciones se utilizan en conjunto para determinar la mejor coincidencia de una ciudad y aplicar un criterio de aceptación basado en el umbral de similitud.
import functools

@functools.lru_cache(maxsize=None)  
def encontrar_mejor_coincidencia(ciudad):
    mejor_coincidencia = process.extractOne(ciudad, ciudades_estados_unidos)
    resultado = mejor_coincidencia[0] if mejor_coincidencia[1] >= calcular_umbral_similitud(len(ciudad)) else ciudad
    return resultado

def calcular_umbral_similitud(longitud_ciudad):
    umbral_base = 55
    umbral = umbral_base - (longitud_ciudad // 3)
    return max(umbral, umbral_base)

In [ ]:
# Probamos la funcion
encontrar_mejor_coincidencia('nw york')

In [ ]:
# Se aplica la función encontrar_mejor_coincidencia a cada valor de la columna 'city' del DataFrame df_b. La función busca la mejor coincidencia de cada ciudad en una lista de ciudades de EE.UU. y actualiza la columna 'city' con las mejores coincidencias encontradas.
# Demora como 17-23 min
df_business2['city'] = df_business2['city'].apply(encontrar_mejor_coincidencia)

In [ ]:
# Generamos un dataframe desde city con las columnas City y State solamente
citystate = city[['city', 'state_name']]

In [ ]:
# Se fusionan dos DataFrames ('df_business' y 'citystate') por la columna 'city' en una unión izquierda.
df_business2 = df_business2.merge(citystate, left_on='city', right_on='city', how='left')

In [ ]:
# Eliminar la columna duplicada 'state'
df_business2.drop('state', axis=1, inplace=True)

In [ ]:
# Se actualiza el dataframe df_business, manteniendo solo las columnas mencionadas.
df_business2 = df_business2[['business_id', 'name', 'address', 'city', 'state_name',
       'postal_code', 'latitude', 'longitude', 'stars', 'review_count',
       'is_open', 'attributes', 'categories', 'hours']]

In [ ]:
# Renombramos la columna state_name a state
df_business2 = df_business2.rename(columns={'state_name':'state'})

In [ ]:
df_business2.head(2)

In [ ]:
# Filtramos el dataframe con los 5 estados que se usarán en el proyecto
# statein = ['New York' , 'Florida' , 'California', 'Nevada' , 'Hawaii']
statein = ['New York' , 'Florida' , 'California', 'Nevada' , 'Hawaii', "Connecticut", "New Jersey", "Texas", "Pennsylvania", "Alaska"]
df_business2 = df_business2[df_business2['state'].isin(statein)].reset_index()

In [ ]:
# print error comprobar huwaii

In [ ]:
df_business2.head(2)

In [ ]:
## Renombramos index
df_business2 = df_business2.rename(columns={'index':'id_business'})

In [ ]:
# Generamos un nuevo dataframe solo con las columnas 'id_business','business_id'
#Creamos la tabla de dimension BusinessYelp
BusinessYelp = df_business2[['id_business', 'business_id']]

# Renombrar la columna "business_id" como "businessYelp_id"
BusinessYelp.rename(columns={"business_id": "businessYelp_id"}, inplace=True)
BusinessYelp.head(2)

In [ ]:
# Exportamos la tabla
#BusinessYelp.to_csv('../Salidas/BusinessYelpId.csv', index=False)

In [ ]:
# Filtramos por ultima vez el dataframe
df_business2 = df_business2[['id_business','name', 'address', 'city', 'state',
       'postal_code', 'latitude', 'longitude', 'stars', 'review_count',
       'is_open', 'attributes', 'categories', 'hours']]
df_business2.head(2)

In [ ]:
# Convertimos a númerico la columna postal_code
df_business2["postal_code"] = df_business2["postal_code"].apply(pd.to_numeric, errors='coerce')

In [ ]:
# Creamos un dataframe auxiliar  que contiene solo las filas donde el valor de la columna 'state' coincide con "New York", "Florida", "California", "Nevada", "Hawaii", "Connecticut", "New Jersey", "Texas", "Pennsylvania" o "Alaska".
aux = df_business2.query('state == "New York" or state == "Florida" or state == "California" or state == "Nevada" or state == "Hawuai" or state == "Connecticut" or state == "New Jerse" or state == "Texas" or state == "Pennsylvania" or state == "Alaska"')

In [ ]:
# Buscamos filas específicas en el DataFrame y realizamos cambios en las columnas 'state' y 'postal_code' en función de los valores de las columnas 'city' y 'postal_code'.
df_business2.loc[df_business2["city"] == "Saint Petersburg", "city"] = "St. Petersburg"
df_business2.loc[(df_business2["city"] == "St. Petersburg") & (df_business2["postal_code"].isnull()), "state"] = "Florida"
df_business2.loc[(df_business2["city"] == "St. Petersburg") & (df_business2["postal_code"].isnull()), "postal_code"] = 33707
df_business2.loc[(df_business2["city"] == "Tampa") & (df_business2["postal_code"].isnull()), "state"] = "Florida"
df_business2.loc[(df_business2["city"] == "Tampa") & (df_business2["postal_code"].isnull()), "postal_code"] = 33610
df_business2.loc[(df_business2["city"] == "Santa Barbara") & (df_business2["postal_code"].isnull()), "state"] = "California"
df_business2.loc[(df_business2["city"] == "Santa Barbara") & (df_business2["postal_code"].isnull()), "postal_code"] = 93101
df_business2.loc[(df_business2["city"] == "Pasco") & (df_business2["postal_code"].isnull()), "state"] = "Florida"
df_business2.loc[(df_business2["city"] == "Pasco") & (df_business2["postal_code"].isnull()), "postal_code"] = 33544

In [ ]:
# Exportamos df_business2 a CSV
#df_business2.to_csv('../Salidas/Dataset_Business.csv', index=False)

In [ ]:
# Exportamos df_business2 a PARQUET
df_business2.to_parquet('../../Data/data_procesada/Dataset_Business.parquet', index=False)

In [ ]:
# Eliminamos algunas columnas del dataframe auxiliar
aux.drop(columns=['is_open'], inplace=True)
aux.drop(columns=['postal_code'], inplace=True)

In [ ]:
# Eliminamos duplicados de la columna auxiliar y reindexamos
aux = aux.drop_duplicates(subset=["id_business", "name"], keep="first")
aux['id_business'] = aux.index
aux.head(2)

In [ ]:
# Generamos un nuevo dataframe con "id_business",'categories'

#Tabla de dimension de categorias
df_cat = aux.loc[:, ['id_business','categories']]

In [ ]:
# Dividir los valores de la columna "categories" por coma y expandirlos en filas
df_cat['categories'] = df_cat['categories'].str.split(',')
df_cat = df_cat[['id_business', 'categories']].explode('categories')

In [ ]:
# Creamos un array con las categorias unicas
categorias = df_cat["categories"].unique()
# Creamos la tabla de dimension de categorias de yelp
df_categorias = pd.DataFrame(categorias, columns=["Descripcion"])
df_categorias['IdCategoria'] = df_categorias.index

In [ ]:
# Exportamos la tabla de dimensiones de categorias
#df_categorias.to_csv("../Salidas/BusinessCategorias.csv", index=False)

In [ ]:
# Creamos una nuevo dataframe auxiliar para el detalle de las categorias a partir de df_Cat y df_categorias
df_aux = df_cat.merge(df_categorias, left_on="categories", right_on="Descripcion", how="inner")

In [ ]:
# Eliminamos columnas categories y Descripcion para generar una tabla de dimension entre id_business y idcategoria
df_aux.drop(columns=["categories"],inplace=True)
df_aux.drop(columns=["Descripcion"],inplace=True)

In [ ]:
# Exportamos tabla de dimensiones de detalle de categorias
#df_aux.to_csv("../Salidas/BusinessDetalleCategorias.csv", index=False)

In [ ]:
# Eliminamos categories de aux
aux.drop(columns=["categories"],inplace=True)

In [ ]:
# Generamos tabla de dimension atributos a partir de aux
df_atribute = aux.loc[:, ["id_business",'attributes']]

In [ ]:
# Procesamos y dividimos los elementos de la columna 'attributes'
for index, i in enumerate(df_atribute['attributes']):
    if isinstance(i,str):
       df_atribute['attributes'][index] = i[1:-1].split(',')

In [ ]:
# Expandimos la columna 'attributes' del DataFrame 'df' en filas separadas, manteniendo los valores correspondientes de la columna 'id_business'.
df_atribute = df_atribute[['id_business', 'attributes']].explode('attributes')

In [ ]:
# Eliminamos los corchetes y comillas dobles de la columna 'attributes' del DataFrame, dejando los valores limpios y sin esos caracteres específicos
df_atribute['attributes'] = df_atribute['attributes'].str.replace('{', '').str.replace('}', '')
df_atribute['attributes'] = df_atribute['attributes'].str.replace('"', '')

In [ ]:
# Modificamos los valores en la columna 'attributes', actualizando ciertos aspectos relacionados con estacionamiento de negocios
df_atribute['attributes'] = df_atribute['attributes'].str.replace('street: True', 'BusinessParking street: True')
df_atribute['attributes'] = df_atribute['attributes'].str.replace('street: False', 'BusinessParking street: False')
df_atribute['attributes'] = df_atribute['attributes'].str.replace('validated: True', 'BusinessParking validated: True')
df_atribute['attributes'] = df_atribute['attributes'].str.replace('validated: False', 'BusinessParking validated: False')
df_atribute['attributes'] = df_atribute['attributes'].str.replace('lot: True', 'BusinessParking lot: True')
df_atribute['attributes'] = df_atribute['attributes'].str.replace('lot: False', 'BusinessParking lot: False')
df_atribute['attributes'] = df_atribute['attributes'].str.replace('valet: True', 'BusinessParking valet: True')
df_atribute['attributes'] = df_atribute['attributes'].str.replace('valet: False', 'BusinessParking valet: False')

In [ ]:
# Creamos un nuevo DataFrame llamado 'df_atributos' que contiene los valores únicos de la columna 'attributes' del DataFrame original 'df_atribute', junto con una columna de identificación única para cada valor.
atributos = df_atribute["attributes"].unique()
df_atributos = pd.DataFrame(atributos, columns=["Descripcion"])
df_atributos['IdAtributos'] = df_atributos.index

In [ ]:
# Exportamos tabla de atributos
#df_atributos.to_csv("../Salidas/BusinessAtributos.csv", index=False)

In [ ]:
# Eliminamos las columnas "attributes" de aux
aux.drop(columns=["attributes"],inplace=True)

In [ ]:
# Creamos la tabla de dimensiones "Hours" a partir de aux
df_hours = aux.loc[:, ["id_business",'hours']]
df_hours

In [ ]:
# Exportamos la tabla de Hours
#df_hours.to_csv("../Salidas/BusinessHoras.csv", index=False)

In [ ]:
# Creamos el dataframe df_Hour_detalle
df_hour_detalle = df_business2[['id_business','hours']]

In [ ]:
df_hour_detalle = df_hour_detalle.explode('hours')
df_hour_detalle

In [ ]:
# Creamos un dataframe llamado hour con las horas unicas de df_hour_detalle
hours = df_hour_detalle['hours'].unique()

In [ ]:
# Se crea un nuevo DataFrame que contiene una sola columna llamada "Descripcion" que contiene los valores de la columna 'hours' y reindexamos
DescHour = pd.DataFrame(hours,columns=["Descripcion"])
DescHour['id_hour'] = DescHour.index

In [ ]:
# Exportamos el dataframe BusinessHorarios
#DescHour.to_csv('../Salidas/BusinessHorarios.csv', index=False)

In [ ]:
# Hacemos una union entre df_hour_detalle y DescHour
df_hour_detalle.merge(DescHour, left_on='hours', right_on='Descripcion', how='inner')[['id_business','id_hour']]

In [ ]:
# Exportamos el dataframe df_hour_detalle
#df_hour_detalle.to_csv('../Salidas/BusinessDetallesHora.csv', index=False)

In [ ]:
# Creamos el dataframe df_detalleatributo a partir de la union entre df_atribute y df_atributos
df_detalleatributo = df_atribute.merge(df_atributos, left_on="attributes", right_on="Descripcion", how="inner")

In [ ]:
# Eliminamos las columnas "attributes" y "Descripcion" de df_detalleatributo
df_detalleatributo.drop(columns = "attributes", inplace=True)
df_detalleatributo.drop(columns = "Descripcion", inplace=True)

In [ ]:
# Exportamos dataframe df_detalleatributo
#df_detalleatributo.to_csv("../Salidas/BusinessDetalleAtributos.csv", index=False)

In [ ]:
# Eliminamos la columna hours de aux
aux.drop(columns=["hours"],inplace=True)
aux.head(2)

## 1.4 tip.json

In [ ]:
# Abrimos el archivo json
json_objects=[]

archivo = os.path.join(Ruta_archivos_Yelp, 'tip.json')
with open(archivo, 'r',encoding='utf-8') as f:
    for line in f:
        json_objects.append(json.loads(line))

df_tip = pd.DataFrame(json_objects)

In [ ]:
# Obtenemos un DataFrame que contenga solo las filas correspondientes a los negocios presentes en BusinessYelp
df_tip = df_tip[df_tip['business_id'].isin(BusinessYelp.businessYelp_id.unique().tolist())]
df_tip

In [ ]:
# Hacemos una unión df_tip con BusinessYelp
df_tip = BusinessYelp.merge(df_tip, left_on='businessYelp_id', right_on='business_id', how='right')[['user_id', 'id_business', 'text', 'date', 'compliment_count']]

In [ ]:
# Hacemos una unión df_tip con UsuarioYelp
df_tip = dfusuario.merge(df_tip, left_on='yelp_id', right_on='user_id', how='right')[['id_user', 'id_business', 'text', 'date', 'compliment_count']]

In [ ]:
# Exportamos a CSV
#df_tip.to_csv('../Salidas/Dataset_Tip.csv', index=False)

In [ ]:
# Exportamos a PARQUET
df_tip.to_parquet('../../Data/data_procesada/Dataset_Tip.parquet', index=False)

## 1.5 review.json

In [ ]:
# Leemos el archivo review.json de Yelp
jsonarr=[]
archivo = os.path.join(Ruta_archivos_Yelp, "review.json")
with open(archivo, 'r',encoding='utf-8') as f:
    for line in f:
        jsonarr.append(json.loads(line))

df_review = pd.DataFrame(jsonarr)

In [ ]:
# Eliminamos duplicados y reindexamos
df_review = df_review.drop_duplicates(subset=["review_id", "text"], keep="first")
df_review['id_review'] = df_review.index

In [ ]:
# Creamos un dataframe con "id_review", "review_id" llamado df_ReviewYelp
df_ReviewYelp = df_review[["id_review", "review_id"]]

In [ ]:
# Exportamos el dataframe ReviewYelpId
#df_ReviewYelp.to_csv("../Salidas/ReviewYelpId.csv", index=False)

In [ ]:
# Eliminamos columna review_id de df_Review
df_review.drop(columns = "review_id",inplace=True)

In [ ]:
# Unimos el dataframe df_review con dfusuario
df_review = df_review.merge(dfusuario, left_on="user_id", right_on="yelp_id", how="inner")

In [ ]:
# Creamos el dataframe df_reviewfinal a partir de df_review y BusinessYelp
df_reviewfinal = df_review.merge(BusinessYelp, left_on="business_id", right_on="businessYelp_id", how="inner")

In [ ]:
# Se eliminan las columnas "user_id" y "yelp_id" del dataframe df_reviewfinal
df_reviewfinal.drop(columns = "business_id", inplace=True)
df_reviewfinal.drop(columns = "businessYelp_id", inplace=True)
df_reviewfinal.drop(columns = "yelp_id", inplace=True)

In [ ]:
df_reviewfinal.head(2)

In [ ]:
# Exportamos a CSV
#df_reviewfinal.to_csv("../Salidas/Dataset_Review.csv", index=False)

In [ ]:
# Exportamos a PARQUET
df_reviewfinal.to_parquet("../../Data/data_procesada/Dataset_Review.parquet")

Datasets exportados disponibles en <https://1drv.ms/f/s!AjBV0Q-Vh1QQmLAZGumu26bVTsnkmw?e=D7EqVJ>

## Extra Convert to parquet

In [ ]:
print error

In [ ]:
import pyarrow.csv as pv
import pyarrow.parquet as pq
import pandas as pd

In [ ]:
table = pv.read_csv("../../Data/data_procesada/Dataset_Business.csv")
pq.write_table(table, "../../Data/data_procesada/Dataset_Business.parquet")

In [ ]:
table = pv.read_csv("../../Data/data_procesada/Dataset_Checkin.csv")
pq.write_table(table, "../../Data/data_procesada/Dataset_Checkin.parquet")

In [ ]:
table = pd.read_csv("../../Data/data_procesada/Dataset_Review.csv")
table.to_parquet("../../Data/data_procesada/Dataset_Review.parquet")

In [ ]:
table = pd.read_csv("../../Data/data_procesada/Dataset_Tip.csv")
table.to_parquet("../../Data/data_procesada/Dataset_Tip.parquet")

In [ ]:
table = pv.read_csv("../../Data/data_procesada/Dataset_User_Elite.csv")
pq.write_table(table, "../../Data/data_procesada/Dataset_User_Elite.parquet")

## Preview de parquets

In [ ]:
table_business = pd.read_parquet("../../Data/data_procesada/Dataset_Business.parquet")
table_business

In [ ]:
null_indices_hora = table_business['hours'].isnull()
null_indices_hora

In [ ]:
random_row = table_business.sample()
random_hours_value = random_row['hours'].values[0]
print(random_hours_value)

In [ ]:
horario_general = {'Friday': '9:0-20:0', 'Monday': '9:0-20:0', 'Saturday': '8:30-15:30', 'Sunday': None, 'Thursday': '9:0-20:0', 'Tuesday': '9:0-20:0', 'Wednesday': '9:0-20:0'}

In [ ]:
table_business.loc[null_indices_hora, 'hours'] = table_business.loc[null_indices_hora].apply(lambda row: horario_general, axis=1)

In [ ]:
null_indices = table_business['hourse'].isnull()
null_indices


In [ ]:
import geocoder

def get_postal_code(latitude, longitude):
    location = geocoder.osm([latitude, longitude], method='reverse')
    return location.postal



In [ ]:
#reemplazo los valores nulos en la tabla
table_business.loc[null_indices, 'postal_code'] = table_business.loc[null_indices].apply(lambda row: get_postal_code(row['latitude'], row['longitude']), axis=1)


In [ ]:

valores_nulos = table_business[table_business['postal_code'].isnull()]
valores_nulos
#este valor nulo me manda al medio del mar por eso no tiene postal code. Considerar si lo borramos o no

In [ ]:
faltante = get_postal_code(32.105261,110.942001)
print(faltante)

In [ ]:
table_business['postal_code'] = table_business['postal_code'].astype(float)
#REVISAR MAÑANA

In [ ]:
duplicates_mask = table_business['address'].duplicated(keep=False)
duplicated_addresses = table_business[duplicates_mask]
duplicated_addresses

In [ ]:
table_business.drop_duplicates(subset=['latitude', 'longitude', 'postal_code'], inplace=True)
table_business

In [ ]:
duplicates_mask = table_business.duplicated(subset=['latitude', 'longitude', 'postal_code'], keep=False)
num_duplicates = duplicates_mask.sum()
num_duplicates

In [ ]:
rows = []

# Iterar sobre cada fila del DataFrame original
for index, row in table_business.iterrows():
    # Obtener las categorías de la fila actual
    categories = row['categories']
    
    # Si hay categorías
    if categories:
        # Dividir las categorías y eliminar espacios en blanco
        categories_list = [category.strip() for category in categories.split(',')]
        
        # Iterar sobre cada categoría y duplicar la fila actual
        for category in categories_list:
            # Crear una copia de la fila actual
            new_row = row.copy()
            # Asignar la categoría actual a la fila copiada
            new_row['categories'] = category
            # Agregar la fila copiada a la lista
            rows.append(new_row)

# Crear un nuevo DataFrame con las filas duplicadas
table_business = pd.DataFrame(rows)


In [ ]:
table_business

In [ ]:
table_business.info()

CHECKIN


In [ ]:
table_checkin = pd.read_parquet("../../Data/data_procesada/Dataset_Checkin.parquet")
table_checkin

In [ ]:
table_checkin = table_checkin.drop_duplicates(subset=['business_id', 'hour', 'date'])

In [ ]:
table_checkin.info()

REVIEWS YELP

In [ ]:
table_reviews = pd.read_parquet("../../Data/data_procesada/Dataset_Review.parquet")
table_reviews

In [ ]:
# Separar los elementos de la columna "date" en cuatro columnas
date_split = table_reviews['date'].str.split(' ', expand=True)

# Asignar nombres a las nuevas columnas
date_split.columns = ['fecha', 'hora']

# Dividir la columna "fecha" en columnas separadas para año, mes y día
date_split[['año', 'mes', 'día']] = date_split['fecha'].str.split('-', expand=True)

# Unir las nuevas columnas con el DataFrame original
table_reviews = table_reviews.join(date_split[['año', 'mes', 'día', 'hora']])

# Eliminar la columna "date"
table_reviews.drop(columns=['date'], inplace=True)


In [ ]:
table_reviews

In [ ]:
# Identificar filas duplicadas basadas en múltiples columnas
#duplicates = table_reviews.duplicated(subset=['user_id', 'text', 'id_user', 'id_business', 'año', 'mes', 'día'], keep=False)
duplicates = table_reviews.duplicated(subset=['user_id', 'id_business', 'año', 'mes', 'día'], keep=False)
# Filtrar el DataFrame original para mostrar solo las filas duplicadas
duplicated_rows = table_reviews[duplicates]

duplicated_rows

In [ ]:
table_reviews = table_reviews.drop_duplicates(subset=['user_id', 'id_business', 'año', 'mes', 'día'])


In [ ]:
table_reviews.info()

TIP

In [ ]:
table_tip = pd.read_parquet("../../Data/data_procesada/Dataset_Tip.parquet")
table_tip

In [ ]:
valores_nulos = table_tip[table_tip['text'].isnull()]
valores_nulos

In [ ]:
duplicates = table_tip.duplicated(subset=['id_user', 'id_business', 'text', 'date'], keep=False)
# Filtrar el DataFrame original para mostrar solo las filas duplicadas
duplicated_rows = table_tip[duplicates]

duplicated_rows

In [ ]:
table_tip = table_tip.drop_duplicates(subset=['id_user', 'id_business', 'text', 'date'])

In [ ]:
table_tip.info()

USER YELP

In [ ]:
table_user = pd.read_parquet("../../Data/data_procesada/Dataset_User_Elite.parquet")
table_user

In [ ]:
duplicates = table_user.duplicated(subset=['id_user'], keep=False)
# Filtrar el DataFrame original para mostrar solo las filas duplicadas
duplicated_rows = table_user[duplicates]

duplicated_rows

In [ ]:
table_user = table_user.drop_duplicates(subset=['id_user'])

In [ ]:
table_user

In [ ]:
table_user.info()